In [ ]:
def google_buildings_csv_to_vector(filename, output=None, **kwargs):
    """
    Convert a CSV file containing Google Buildings data to a GeoJSON vector file.

    Args:
        filename (str): The path to the input CSV file.
        output (str, optional): The path to the output GeoJSON file. If not provided, the output file will have the same
            name as the input file with the extension changed to '.geojson'.
        **kwargs: Additional keyword arguments that are passed to the `to_file` method of the GeoDataFrame.

    Returns:
        None
    """
    import os
    import pandas as pd
    import geopandas as gpd
    from shapely import wkt

    df = pd.read_csv(filename)

    # Create a geometry column from the "geometry" column in the DataFrame
    df["geometry"] = df["geometry"].apply(wkt.loads)

    # Convert the pandas DataFrame to a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry="geometry")
    gdf.crs = "EPSG:4326"

    if output is None:
        output = os.path.splitext(filename)[0] + ".geojson"

    gdf.to_file(output, **kwargs)

For example, download the Google buildings footprint of area impacted by flooding in Derna, Libya from https://data.humdata.org/dataset/buildings-footprint-floods-derna-libya

In [ ]:
url = 'https://data.humdata.org/dataset/cf84d369-eaba-410e-8e38-778eeb238f71/resource/b8dcedcc-614f-45c4-8618-d3c89a6c36b6/download/open_buildings_v3_polygons_your_own_wkt_polygon_derna.csv.gz'

In [ ]:
google_buildings_csv_to_vector(url, output='buildings.geojson')

In [ ]:
import leafmap.foliumap as leafmap

In [ ]:
m = leafmap.Map(height='700px')
# Add Google satellite basemap
m.add_basemap('SATELLITE')
# Add post-flood image from Maxar Open Data
image = 'https://raw.githubusercontent.com/opengeos/maxar-open-data/master/datasets/Libya-Floods-Sept-2023/10500100363D0900.json'
m.add_stac_layer(image, name='Maxar Open Data')
# Add Google Buildings
m.add_vector('buildings.geojson', layer_name='Google Buildings', zoom_to_layer=True, info_mode='on_click')
m